In [30]:
import glob 
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import cv2
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
import csv 
from PIL import Image
import pandas as pd
import shutil
import ast

In [9]:
df_r = pd.read_csv('R-Peaks1.csv')
df = pd.read_csv('labels(inc).csv')


In [10]:
df_r

,name,esm
0,6fwt32TiT_KD-QU8t64_eg 65,"[2, 28, 77]"
1,6fwt32TiT_KD-QU8t64_eg 21,"[1, 13, 30, 35]"
2,6fwt32TiT_KD-QU8t64_eg 46,"[1, 40, 80, 117]"
3,6fwt32TiT_KD-QU8t64_eg 56,"[2, 42, 80]"
4,6fwt32TiT_KD-QU8t64_eg 11,"[1, 37]"
...,...,...
5298,0oMWy6kqR0edXPJd8BbxYg 22,"[0, 0]"
5299,0oMWy6kqR0edXPJd8BbxYg 5,"[1, 13, 27]"
5300,0oMWy6kqR0edXPJd8BbxYg 74,"[1, 31]"
5301,0oMWy6kqR0edXPJd8BbxYg 29,"[2, 42, 71]"


In [11]:
df.label.value_counts()

Other                640
Apical-4ch           409
PLAX                 407
Subcostal            288
Apical-2ch           192
PSAX-mid             188
Apical-3ch           177
Apical-4ch-d         177
PSAX-ves             176
Other-d              170
PSAX-ves-d           166
PLAX-d               163
Subcostal-d          149
PSAX-base            144
Subcostal-2D-d       143
PSAX-apical          123
PSAX-ves-2D-d        120
Apical-3ch-d         112
Apical-5ch            94
PLAX-inflow           88
Apical-5ch-d          82
Other-2D-d            77
Apical-2ch-d          76
PLAX-inflow-d         75
Suprasternal-2D-d     67
Apical-4ch-2D-d       66
PLAX-outflow          59
Suprasternal          58
PLAX-outflow-d        57
Contrast-4ch          50
Apical-3ch-2D-d       48
PLAX-2D-d             47
Suprasternal-d        43
Apical-5ch-2D-d       35
Contrast-2ch          35
Contrast-3ch          35
PSAX-base-2D-d        24
Apical-2ch-2D-d       24
PLAX-inflow-2D-d      18
PLAX-outflow-2D-d     16


In [12]:
print("Before NaN drop", len(df))
df = df.dropna(subset='label')
print("After NaN drop", len(df))
df['label'] = df['label'].apply(lambda x: x.split("-d")[0])

Before NaN drop 5303
After NaN drop 5140


In [13]:
df.label.value_counts()

Other              810
Apical-4ch         586
PLAX               570
Subcostal          437
PSAX-ves           342
Apical-3ch         289
Apical-2ch         268
PSAX-mid           189
Apical-5ch         176
PLAX-inflow        163
PSAX-base          155
Subcostal-2D       143
PSAX-apical        125
PSAX-ves-2D        120
PLAX-outflow       116
Suprasternal       101
Other-2D            77
Suprasternal-2D     67
Apical-4ch-2D       66
Contrast-4ch        50
Apical-3ch-2D       48
PLAX-2D             47
Apical-5ch-2D       35
Contrast-2ch        35
Contrast-3ch        35
PSAX-base-2D        24
Apical-2ch-2D       24
PLAX-inflow-2D      18
PLAX-outflow-2D     16
PSAX-mid-2D          3
PSAX-apical-2D       3
PSAX-ves-base        1
Other                1
Name: label, dtype: int64

In [7]:
# df['label'] = df['label'].apply(lambda x: x.split('-2D')[0])

In [14]:
df.label.value_counts()

Other              810
Apical-4ch         586
PLAX               570
Subcostal          437
PSAX-ves           342
Apical-3ch         289
Apical-2ch         268
PSAX-mid           189
Apical-5ch         176
PLAX-inflow        163
PSAX-base          155
Subcostal-2D       143
PSAX-apical        125
PSAX-ves-2D        120
PLAX-outflow       116
Suprasternal       101
Other-2D            77
Suprasternal-2D     67
Apical-4ch-2D       66
Contrast-4ch        50
Apical-3ch-2D       48
PLAX-2D             47
Apical-5ch-2D       35
Contrast-2ch        35
Contrast-3ch        35
PSAX-base-2D        24
Apical-2ch-2D       24
PLAX-inflow-2D      18
PLAX-outflow-2D     16
PSAX-mid-2D          3
PSAX-apical-2D       3
PSAX-ves-base        1
Other                1
Name: label, dtype: int64

In [15]:
valid_labels = {'PLAX', 'PSAX-ves', 'PSAX-base', 'PSAX-mid', 'PSAX-apical', 'Apical-2ch', 'Apical-3ch', 'Apical-5ch', 'Apical-4ch', 'Suprasternal', 'Subcostal'}
df['label'] = df['label'].apply(lambda x : x if x in valid_labels else "other")

In [16]:
df.label.value_counts()

other           1902
Apical-4ch       586
PLAX             570
Subcostal        437
PSAX-ves         342
Apical-3ch       289
Apical-2ch       268
PSAX-mid         189
Apical-5ch       176
PSAX-base        155
PSAX-apical      125
Suprasternal     101
Name: label, dtype: int64

In [17]:
df = df[df['label'] != 'other'].reset_index()

In [18]:
df.drop('index', axis=1, inplace= True)

In [19]:
df

,folder_name,file_name,label,comments,combined
0,_3wRyUbdSGmjEWULRVg-jw,3,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 3
1,_3wRyUbdSGmjEWULRVg-jw,4,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 4
2,_3wRyUbdSGmjEWULRVg-jw,5,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 5
3,_3wRyUbdSGmjEWULRVg-jw,7,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 7
4,_3wRyUbdSGmjEWULRVg-jw,9,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 9
...,...,...,...,...,...
3233,6VLObFIMTzq7fyUz9-ltQA,74,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 74
3234,6VLObFIMTzq7fyUz9-ltQA,75,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 75
3235,6VLObFIMTzq7fyUz9-ltQA,77,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 77
3236,6VLObFIMTzq7fyUz9-ltQA,78,Suprasternal,NaN,6VLObFIMTzq7fyUz9-ltQA 78


In [20]:
df.rename(columns={'combined': 'name'}, inplace=True)

In [21]:
df

,folder_name,file_name,label,comments,name
0,_3wRyUbdSGmjEWULRVg-jw,3,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 3
1,_3wRyUbdSGmjEWULRVg-jw,4,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 4
2,_3wRyUbdSGmjEWULRVg-jw,5,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 5
3,_3wRyUbdSGmjEWULRVg-jw,7,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 7
4,_3wRyUbdSGmjEWULRVg-jw,9,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 9
...,...,...,...,...,...
3233,6VLObFIMTzq7fyUz9-ltQA,74,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 74
3234,6VLObFIMTzq7fyUz9-ltQA,75,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 75
3235,6VLObFIMTzq7fyUz9-ltQA,77,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 77
3236,6VLObFIMTzq7fyUz9-ltQA,78,Suprasternal,NaN,6VLObFIMTzq7fyUz9-ltQA 78


In [22]:
df_r

,name,esm
0,6fwt32TiT_KD-QU8t64_eg 65,"[2, 28, 77]"
1,6fwt32TiT_KD-QU8t64_eg 21,"[1, 13, 30, 35]"
2,6fwt32TiT_KD-QU8t64_eg 46,"[1, 40, 80, 117]"
3,6fwt32TiT_KD-QU8t64_eg 56,"[2, 42, 80]"
4,6fwt32TiT_KD-QU8t64_eg 11,"[1, 37]"
...,...,...
5298,0oMWy6kqR0edXPJd8BbxYg 22,"[0, 0]"
5299,0oMWy6kqR0edXPJd8BbxYg 5,"[1, 13, 27]"
5300,0oMWy6kqR0edXPJd8BbxYg 74,"[1, 31]"
5301,0oMWy6kqR0edXPJd8BbxYg 29,"[2, 42, 71]"


In [23]:
df_r = df_r[df_r['name'].isin(df['name'])].reset_index()

In [24]:
df_r.drop(['index'], axis=1, inplace=True)

In [25]:
df_r.to_csv('filtered_Rpeaks.csv', index=False)
df.to_csv('filtered_labels.csv', index=False)

In [26]:
rp = pd.read_csv('filtered_Rpeaks.csv')
lbl = pd.read_csv('filtered_labels.csv')

In [27]:
rp

,name,esm
0,6fwt32TiT_KD-QU8t64_eg 65,"[2, 28, 77]"
1,6fwt32TiT_KD-QU8t64_eg 46,"[1, 40, 80, 117]"
2,6fwt32TiT_KD-QU8t64_eg 47,"[1, 23, 34]"
3,6fwt32TiT_KD-QU8t64_eg 83,"[3, 21, 30]"
4,6fwt32TiT_KD-QU8t64_eg 79,"[0, 9, 31]"
...,...,...
3233,0oMWy6kqR0edXPJd8BbxYg 8,"[3, 39]"
3234,0oMWy6kqR0edXPJd8BbxYg 57,"[2, 32]"
3235,0oMWy6kqR0edXPJd8BbxYg 33,"[1, 14, 24]"
3236,0oMWy6kqR0edXPJd8BbxYg 74,"[1, 31]"


In [28]:
lbl

,folder_name,file_name,label,comments,name
0,_3wRyUbdSGmjEWULRVg-jw,3,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 3
1,_3wRyUbdSGmjEWULRVg-jw,4,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 4
2,_3wRyUbdSGmjEWULRVg-jw,5,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 5
3,_3wRyUbdSGmjEWULRVg-jw,7,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 7
4,_3wRyUbdSGmjEWULRVg-jw,9,Subcostal,NaN,_3wRyUbdSGmjEWULRVg-jw 9
...,...,...,...,...,...
3233,6VLObFIMTzq7fyUz9-ltQA,74,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 74
3234,6VLObFIMTzq7fyUz9-ltQA,75,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 75
3235,6VLObFIMTzq7fyUz9-ltQA,77,Subcostal,NaN,6VLObFIMTzq7fyUz9-ltQA 77
3236,6VLObFIMTzq7fyUz9-ltQA,78,Suprasternal,NaN,6VLObFIMTzq7fyUz9-ltQA 78


In [31]:
rp['peaks'] = rp[' esm'].astype(str).apply(ast.literal_eval)
rp['peaks'] = rp['peaks'].apply(lambda x: x[1:])

In [35]:
rp['freq_count'] = rp['peaks'].apply(lambda x: len(x)-1)

In [41]:
res = pd.merge(lbl, rp, on='name').drop(['comments','name', ' esm' ], axis=1)

In [56]:
res

,folder_name,file_name,label,peaks,freq_count
0,_3wRyUbdSGmjEWULRVg-jw,3,Subcostal,"[17, 33]",1
1,_3wRyUbdSGmjEWULRVg-jw,4,Subcostal,"[86, 104]",1
2,_3wRyUbdSGmjEWULRVg-jw,5,Subcostal,"[9, 15, 22]",2
3,_3wRyUbdSGmjEWULRVg-jw,7,Subcostal,"[49, 87]",1
4,_3wRyUbdSGmjEWULRVg-jw,9,Subcostal,"[11, 31]",1
...,...,...,...,...,...
3233,6VLObFIMTzq7fyUz9-ltQA,74,Subcostal,[22],0
3234,6VLObFIMTzq7fyUz9-ltQA,75,Subcostal,"[12, 26]",1
3235,6VLObFIMTzq7fyUz9-ltQA,77,Subcostal,"[16, 27]",1
3236,6VLObFIMTzq7fyUz9-ltQA,78,Suprasternal,"[40, 78]",1


In [54]:
res['freq_count'].value_counts()

1    1838
2     718
0     682
Name: freq_count, dtype: int64

In [55]:
res.

,folder_name,file_name,label,peaks,freq_count
0,_3wRyUbdSGmjEWULRVg-jw,3,Subcostal,"[17, 33]",1
1,_3wRyUbdSGmjEWULRVg-jw,4,Subcostal,"[86, 104]",1
2,_3wRyUbdSGmjEWULRVg-jw,5,Subcostal,"[9, 15, 22]",2
3,_3wRyUbdSGmjEWULRVg-jw,7,Subcostal,"[49, 87]",1
4,_3wRyUbdSGmjEWULRVg-jw,9,Subcostal,"[11, 31]",1
...,...,...,...,...,...
3233,6VLObFIMTzq7fyUz9-ltQA,74,Subcostal,[22],0
3234,6VLObFIMTzq7fyUz9-ltQA,75,Subcostal,"[12, 26]",1
3235,6VLObFIMTzq7fyUz9-ltQA,77,Subcostal,"[16, 27]",1
3236,6VLObFIMTzq7fyUz9-ltQA,78,Suprasternal,"[40, 78]",1


In [74]:
new_data = []

for idx, row in res.iterrows():
    for count in range(row['freq_count'] + 1):
        new_row = row.copy()
        new_row['sub_dir'] = count
        new_data.append(new_row)


In [81]:
new_data = pd.DataFrame(new_data).reset_index(drop=True)

In [85]:
new_data.drop(['freq_count', 'peaks'], axis=1, inplace=True)

In [91]:
new_data = new_data.reindex(['folder_name','file_name','sub_dir','label'], axis=1)

In [92]:
new_data

,folder_name,file_name,sub_dir,label
0,_3wRyUbdSGmjEWULRVg-jw,3,0,Subcostal
1,_3wRyUbdSGmjEWULRVg-jw,3,1,Subcostal
2,_3wRyUbdSGmjEWULRVg-jw,4,0,Subcostal
3,_3wRyUbdSGmjEWULRVg-jw,4,1,Subcostal
4,_3wRyUbdSGmjEWULRVg-jw,5,0,Subcostal
...,...,...,...,...
6507,6VLObFIMTzq7fyUz9-ltQA,77,1,Subcostal
6508,6VLObFIMTzq7fyUz9-ltQA,78,0,Suprasternal
6509,6VLObFIMTzq7fyUz9-ltQA,78,1,Suprasternal
6510,6VLObFIMTzq7fyUz9-ltQA,79,0,Suprasternal


In [93]:
new_data.to_csv('new_label_with_rest_peaks.csv', index=False)

### continues on filter ECG effcicient writing 